In [1]:
import re

# 開啟儲存過程檔案
with open('agdsp/StoredProcedures/uspAccidentFormTemplateGet.sql','r', encoding='utf-8') as file:
    contents = file.read()

# print(contents)

# 提取儲存過程名稱
procedure_name = re.findall(r'CREATE\sPROCEDURE\s(.+)\(', contents)[0]

# 提取輸入參數
input_params = re.findall(r'@[\w]+\s[\w]+\s(,\n\s+@[\w]+\s[\w]+\s)*', contents)[0]

# 提取輸出參數
output_params = re.findall(r'OUTPUT\s@[\w]+\s[\w]+', contents)

# 提取註解
comments = re.findall(r'/\*[\s\S]*?\*/', contents)

# 將提取到的信息寫入到Word檔案中
from docx import Document
from docx.shared import Inches

document = Document()
document.add_heading(procedure_name, 0)

document.add_heading('Input Parameters', level=1)
document.add_paragraph(input_params)

if output_params:
    document.add_heading('Output Parameters', level=1)
    for param in output_params:
        document.add_paragraph(param)

if comments:
    document.add_heading('Comments', level=1)
    for comment in comments:
        document.add_paragraph(comment)

document.save('your_word_file.docx')


In [137]:
import re
from docx import Document
from docx.shared import Cm

def set_text(cell, text):
    cell.text = text

# 讀取 SQL 檔案
with open('agdsp/StoredProcedures/uspUserRoleHistQuery.sql', 'r', encoding='utf-8') as f:
    sql = f.read()

# 定義正則表達式
return_pattern = re.compile(r'\*\* Return Recordset: \n(.*?)\*\*\n', re.DOTALL)
spName_pattern = re.compile(r'\*\* Name: (.*?)\n', re.DOTALL)
spDscr_pattern = re.compile(r'\*\* Desc: (.*?)\n', re.DOTALL)
input_pattern = re.compile(r'\*\*\tInput\n\*\* -----------*\n(.*?)\*\*\n', re.DOTALL)
output_pattern = re.compile(r'\*\*   Output\n\*\* -----------*\n(.*?)\*\* \n', re.DOTALL)


# 抽取變數信息
sp_Name, sp_Dscr = spName_pattern.findall(sql), spDscr_pattern.findall(sql)
input_vars = input_pattern.findall(sql)
output_vars = output_pattern.findall(sql)
return_Rows = return_pattern.findall(sql)

print(return_Rows)
# print(table_Dscr)
# print(1,input_vars)
# print(2,output_vars)
# 創建 Word 文件
doc = Document()

table1 = doc.add_table(rows = 1, cols = 4)
table1.style = 'Table Grid'


cells = table1.rows[0].cells
line1 = ['預存程序', sp_Name, '作者', 'Unisys']
list(map(lambda h: set_text(cells[line1.index(h)], h), line1))

m_cells2 = table1.add_row().cells
m_cells2[0].text = '程序描述'
m_cells2[1].text = table_Dscr
m_cells2[1].merge(m_cells2[2]).merge(m_cells2[3])

# 創建表格
table = doc.add_table(rows = 1, cols = 6)
table.style = 'Table Grid'

# 添加表頭
header = ['序號','輸入參數','輸入說明','資料型態','長度', 'In/Out']
hdr_cells = table.rows[0].cells
list(map(lambda h: set_text(hdr_cells[header.index(h)], h), header))

# 添加變數信息
print('cc', input_vars)
inputs = input_vars[0].split('\n\t')
i = 0
for var_info in inputs:
    list2 = re.split(r'[-,\s,\t]\s*', var_info.strip())
    var, dataType, desc = [e for e in list2 if e != ""]
    print(var, dataType, desc)
    dataTypes = re.split(r'[(,)]', dataType) 

    row_cells = table.add_row().cells
    i += 1
    row_cells[0].text = str(i)
    row_cells[1].text = var
    row_cells[2].text = desc
    row_cells[3].text = dataTypes[0]
    row_cells[4].text = dataTypes[1] if (len(dataTypes) >= 2) else '' 
    row_cells[5].text = 'input'


# 輸出參數 ###
header = ['序號','輸出參數','輸出說明','資料型態','長度', 'In/Out']
hdr_cells = table.add_row().cells
list(map(lambda h: set_text(hdr_cells[header.index(h)], h), header))
i = 0
for var_info in output_vars:
    list2 = re.split(r'[-,\s,\t]\s*', var_info.strip())
    var, dataType, desc = [e for e in list2 if e != ""]
    print(var, dataType, desc)
    dataTypes = re.split(r'[(,)]', dataType) 
    row_cells = table.add_row().cells
    i += 1
    row_cells[0].text = str(i)
    row_cells[1].text = var
    row_cells[2].text = desc
    row_cells[3].text = dataTypes[0]
    row_cells[4].text = dataTypes[1] if (len(dataTypes) >= 2) else '' 
    row_cells[5].text = 'output'


# 輸出參數 ###
header = ['序號','產出欄位','欄位說明','資料型態','長度', 'In/Out']
hdr_cells = table.add_row().cells
list(map(lambda h: set_text(hdr_cells[header.index(h)], h), header))
i = 0
returnRows = return_Rows[0].split('\n')
print(returnRows)
for row in returnRows:
    list2 = re.split(r'[-,\s,\t]\s*', row.strip())
    li = [e for e in list2 if e != ""]
    if len(li) > 0:
        ff, var, dataType, desc = li
        print(ff, var, dataType, desc)
        dataTypes = re.split(r'[(,)]', dataType) 
        row_cells = table.add_row().cells
        i += 1
        row_cells[0].text = str(i)
        row_cells[1].text = var
        row_cells[2].text = desc
        row_cells[3].text = dataTypes[0]
        row_cells[4].text = dataTypes[1] if (len(dataTypes) >= 2) else '' 
        row_cells[5].text = ''

doc.save('storeProcedureReport.docx')

#  由MSSQL找出對應sp的輸出入參數
# SELECT 
#    SCHEMA_NAME(SCHEMA_ID) AS [Schema]
#   ,SO.name AS [ObjectName]      
#   ,SO.Type_Desc AS [ObjectType (UDF/SP)]
#   ,P.parameter_id AS [ParameterID]
#   ,P.name AS [ParameterName]
#   ,TYPE_NAME(P.user_type_id) AS [ParameterDataType]
#   ,P.max_length AS [ParameterMaxBytes]
#   ,P.is_output AS [IsOutPutParameter]
# FROM sys.objects AS SO
# INNER JOIN sys.parameters AS P ON SO.OBJECT_ID = P.OBJECT_ID
# ORDER BY [Schema], SO.name, P.parameter_id
# GO

['** SeqNo            INT\t\t\t\t- 流水號\n** UserID           VARCHAR(11)\t\t- 員工編號\n** RoleID           VARCHAR(20)\t\t- 角色ID\n** CreateDT         DATETIME2(7)\t- 建立時間\n** Creator          VARCHAR(11)\t\t- 建立者\n** CreatorName      NVARCHAR(60)\t- 建立人員\n** UpdateDT         DATETIME2(7)\t- 更新時間\n** Updater          VARCHAR(11)\t\t- 更新者\n** UpdaterName      NVARCHAR(60)\t- 更新人員\n** ActionType\t\tCHAR(1)\t\t\t- 執行動作\n** ActionDT\t\t\tDATETIME2(7)\t- 執行動作時間\n** Actor\t\t\tVARCHAR(11)\t\t- 執行動作者\n** ActorName\t\tNVARCHAR(60)\t- 執行動作人員\n** Total            INT\t\t\t\t- 資料總筆數\n']
cc ['\t@StartTime\t\tDATETIME2\t\t- 開始時間\n\t@EndTime\t\tDATETIME2\t\t- 結束時間\n\t@Actor\t\t\tVARCHAR(11)\t\t- 執行動作人員\n\t@Page \t\t\tINT \t\t\t- 頁數\n\t@RowsPerPage \tINT \t\t\t- 每頁筆數\n\t@SortColumn \tNVARCHAR(30) \t- 排序欄位\n\t@SortOrder \t\tVARCHAR(10) \t- 排序順序\n']
@StartTime DATETIME2 開始時間
@EndTime DATETIME2 結束時間
@Actor VARCHAR(11) 執行動作人員
@Page INT 頁數
@RowsPerPage INT 每頁筆數
@SortColumn NVARCHAR(30) 排序欄位
@SortOrder VARCHAR(

In [11]:
from docx import Document
from docx.enum.table import WD_ALIGN_VERTICAL

document = Document()

# 創建一個 table，有 2 個 row 和 2 個 column
table = document.add_table(rows=2, cols=2)

# 取得第一個 cell
cell_1_1 = table.cell(0, 0)

# 設定 cell 的垂直對齊方式為中央對齊
cell_1_1.vertical_alignment = WD_ALIGN_VERTICAL.CENTER

# 寫入文字到第一個 cell
cell_1_1.text = 'This cell will span two rows'

# 取得第二個 cell
cell_2_1 = table.cell(1, 0)

# 寫入文字到第二個 cell
cell_2_1.text = 'This is the second cell'

# 合併第一個 cell 和第二個 cell
cell_1_1.merge(cell_2_1)

# 儲存文件
document.save('table11.docx')